In [26]:
from dotenv import load_dotenv


load_dotenv()

True

In [1]:
from src.agent import Agent

agent = Agent()
agent.viz_graph(ipynb=True)

I0000 00:00:1728316657.378803 10751030 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1728316657.541883 10751030 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os
import logging
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def search_by_image(image_path):
    # Setup Chrome options
    chrome_options = webdriver.ChromeOptions()
    # Uncomment the line below if you want to run in headless mode
    chrome_options.add_argument("--headless")  # Enable headless mode
    chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
    chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

    
    # Initialize the driver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), 
                            options=chrome_options)
    
        # Navigate to Google Images
    driver.get("https://images.google.com/?hl=en")
    logging.info("Navigated to Google Lens")

    accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'L2AGLb')))  # Using the button's ID)
    accept_button.click()

    camera_icon = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//div[@aria-label="Search by image" and @role="button"]')))
    camera_icon.click()

    # input
    file_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='file']"))
    )
    
    
    # Convert to absolute path if needed
    abs_image_path = os.path.abspath(image_path)
    file_input.send_keys(abs_image_path)
    
    # Wait for the search results to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.ksQYvb'))  # Use the updated class name
    )

    # Locate the divs containing the image results
    results = driver.find_elements(By.CSS_SELECTOR, 'div.ksQYvb')  # Find the correct class name for results

    valid_results = []
    # Extract and print the data attributes and image URLs
    for result in results:
        # Get the data attributes
        title = result.get_attribute('data-item-title')
        action_url = result.get_attribute('data-action-url')

        # Only consider results with a valid title
        if title:
            # Get the image URL
            try:
                image_div = result.find_element(By.CSS_SELECTOR, 'div.Me0cf')  # Locate the image div
                img = image_div.find_element(By.TAG_NAME, 'img')  # Locate the image tag
                img_src = img.get_attribute('src')  # Get the source URL of the image
            except Exception as e:
                img_src = "No image found"

            # Append to valid results
            valid_results.append({
                'title': title,
                'action_url': action_url,
                'image_url': img_src
            })

    # Display only the top 5 results
    for i, result in enumerate(valid_results[:5]):
        print(f"Result {i + 1}:\nTitle: {result['title']}\nAction URL: {result['action_url']}\nImage URL: {result['image_url']}\n")

    driver.quit()
    
    
image_path = "1.jpg"  # Replace with your image path
search_by_image(image_path)




2024-10-08 13:02:27,344 - INFO - Validated image path: /workspaces/GeoAI/test/test_images/test.jpg
2024-10-08 13:02:27,346 - INFO - ====== WebDriver manager ======
2024-10-08 13:02:27,365 - INFO - Get LATEST chromedriver version for google-chrome
2024-10-08 13:02:27,389 - INFO - About to download new driver from https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
2024-10-08 13:02:27,409 - INFO - Driver downloading response is 200


2024-10-08 13:02:27,486 - INFO - Get LATEST chromedriver version for google-chrome
2024-10-08 13:02:27,667 - INFO - Get LATEST chromedriver version for google-chrome
2024-10-08 13:02:27,696 - INFO - Driver has been saved in cache [/home/codespace/.wdm/drivers/chromedriver/linux64/114.0.5735.90]
2024-10-08 13:02:27,789 - ERROR - Failed to create driver: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x5b2f43c614e3 <unknown>
#1 0x5b2f43990c76 <unknown>
#2 0x5b2f439b7757 <unknown>
#3 0x5b2f439b6029 <unknown>
#4 0x5b2f439f4ccc <unknown>
#5 0x5b2f439f447f <unknown>
#6 0x5b2f439ebde3 <unknown>
#7 0x5b2f439c12dd <unknown>
#8 0x5b2f439c234e <unknown>
#9 0x5b2f43c213e4 <unknown>
#10 0x5b2f43c253d7 <unknown>
#11 0x5b2f43c2fb20 <unknown>
#12 0x5b2f43c26023 <unknown>
#13 0x5b2f43bf41aa <unknown>
#14 0x5b2f43c4a6b8 <unknown>
#15 0x5b2f43c4a847 <unknown>
#16 0x5b2f43c5a243 <unknown>
#17 0x758593e63609 start_thread

2024-10-08 13:02:27,790 - ERROR - Search failed: Message: unknown e

Error: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x5b2f43c614e3 <unknown>
#1 0x5b2f43990c76 <unknown>
#2 0x5b2f439b7757 <unknown>
#3 0x5b2f439b6029 <unknown>
#4 0x5b2f439f4ccc <unknown>
#5 0x5b2f439f447f <unknown>
#6 0x5b2f439ebde3 <unknown>
#7 0x5b2f439c12dd <unknown>
#8 0x5b2f439c234e <unknown>
#9 0x5b2f43c213e4 <unknown>
#10 0x5b2f43c253d7 <unknown>
#11 0x5b2f43c2fb20 <unknown>
#12 0x5b2f43c26023 <unknown>
#13 0x5b2f43bf41aa <unknown>
#14 0x5b2f43c4a6b8 <unknown>
#15 0x5b2f43c4a847 <unknown>
#16 0x5b2f43c5a243 <unknown>
#17 0x758593e63609 start_thread

